In [1]:
import torch
from torch import nn

from data_loading import load_dataset

from core import train, test

In [2]:
BATCH_SIZE = 64
DATASET_SIZE = 800

In [3]:
dataset = load_dataset('games_dataset.csv', BATCH_SIZE)
train_dataset = dataset[:DATASET_SIZE]
test_dataset = dataset[DATASET_SIZE:]

In [4]:
def accuracy(out, truth):
    return torch.abs(truth - out)

In [5]:
class Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(*args, **kwargs)

    def forward(self, X):
        return self.classifier.forward(X)


model = Model(nn.Conv2d(8, 32, kernel_size=4, stride=2, padding=4),
              nn.ReLU(),
              nn.Conv2d(32, 128, kernel_size=6, stride=2, padding=4),
              nn.ReLU(),
              nn.Conv2d(128, 512, kernel_size=6, stride=2, padding=2),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=1),
              nn.Conv2d(512, 64, kernel_size=1, stride=2),
              nn.ReLU(),
              nn.Flatten(),
              nn.Linear(64, 16),
              nn.ReLU(),
              nn.Linear(16, 1),
              )

In [6]:
train(train_dataset,
      BATCH_SIZE,
      model,
      nn.MSELoss(),
      torch.optim.SGD(model.classifier.parameters(), lr=0.02),
      accuracy,
      60)

Dataset size: 800
Epoch [1/60], train_loss: 12.095031643509865, train_accuracy: 2.6194998218119143, time: 1.441s
Epoch [2/60], train_loss: 11.266295641064644, train_accuracy: 2.550138873755932, time: 1.207s
Epoch [3/60], train_loss: 7.70222268551588, train_accuracy: 2.0847493118047713, time: 1.171s
Epoch [4/60], train_loss: 5.697461593747139, train_accuracy: 1.7648500618338585, time: 1.213s
Epoch [5/60], train_loss: 4.353302395045757, train_accuracy: 1.52568773239851, time: 1.214s
Epoch [6/60], train_loss: 3.1159750169515608, train_accuracy: 1.2800856804847718, time: 1.164s
Epoch [7/60], train_loss: 2.263608800545335, train_accuracy: 1.086095750182867, time: 1.156s
Epoch [8/60], train_loss: 1.7289207847416401, train_accuracy: 0.9443839327245951, time: 1.18s
Epoch [9/60], train_loss: 1.3629781926423312, train_accuracy: 0.834900007173419, time: 1.165s
Epoch [10/60], train_loss: 1.1152136020734906, train_accuracy: 0.7565511754155159, time: 1.178s
Epoch [11/60], train_loss: 0.8921486155316

In [7]:
test(test_dataset, BATCH_SIZE, model, nn.MSELoss(), accuracy)

Dataset size: 131
test_loss: 1.3569791867532803, test_accuracy: 0.7366140514384699, time: 0.059s


In [8]:
test(train_dataset, BATCH_SIZE, model, nn.MSELoss(), accuracy)

Dataset size: 800
test_loss: 0.10407889988040551, test_accuracy: 0.2333738158456981, time: 0.349s
